`GB` BigData / [Олег Гладкий](https://gb.ru/users/3837199) // домашнее задание

`262698` __Методы сбора и обработки данных из сети Интернет__:  `08`. Scrapy + API

# Scrapy + API: визуализация

Визуализация данных, полученных Scrapy при помощи API

## MongoDB

__Соединяемся__ с базой данных вакансий, полученных в результат работы паука `hh_api.py`

In [1]:
import pandas as pd
from pprint import pprint
import pymongo
import sys

1. __Сервер__: соединяемся с сервером MongoDB: инициализируем клиента соединения `client`.
2. __БД__: подключаемся к базе данных `job_api` инициализируя переменную `db`
3. __Коллекция__: инициализируем ссылку `db` на коллекцию `vacancies_api`

In [2]:
client = pymongo.MongoClient('mongodb://127.0.0.1:27017') 
db = client.job_api
vacancies = db.vacancies_api

__Проверяем__: _точно-ли в нашей беза имеется коллекция_

In [3]:
db.list_collection_names()

['vacancies_api']

__Индекс__: сортировка по дате — создадим обычный индекс по полю `vacancies_api`. (У меня у меня mongo 4.2.23.)

Инфо: https://pymongo.readthedocs.io/en/stable/tutorial.html

In [4]:
index_date = vacancies.create_index([('vacancies_api', pymongo.ASCENDING)], unique=False)
index_date, type(index_date)

('vacancies_api_1', str)

## Запрос вакансий по критерию

Выведем вакансии, соответствующие заданному уровню зарплаты. Внимание! Необходимо анализировать оба поля зарплаты.

__Задайте уровень дохода__

Необходимо задать:
* уровень дохода `pay_level_ru` в рублях

А так же определить курсы валют: 
* курс доллара `rate_us_dollar`
* курс евро `rate_euro`

Полученная выбрка вакансий будет (в том числе) выведена в файл `04_hw_MongoDB_Python__pay_selection.txt` на диск.

In [5]:
pay_level_ru = 150000       # Задайте зарплату

rate_us_dollar = 70         # Курс доллара 
rate_euro = 70              # Курс евро


# Пересчёт уровня в рубли для "долларовых" вакансий

pay_level_us = pay_level_ru / rate_us_dollar
pay_level_eu = pay_level_ru / rate_euro

curr_ru='руб'
curr_us='USD'
curr_eu='EUR'

__Валюты__, проверка: проверяем валюты зарплатных предложений: выводим только уникальные значения поля `Maney_curr`

In [6]:
vacancies.distinct('salary_cur')

['EUR', 'RUR', 'USD']

__Формирование запроса__

Сформируем запрос для получения выборки (в виде курсора на эти данные). При этом учтём соглашения, принятые для соответствия данным сайта и значениям словаря на примере:
* сайт: от 1000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 0
* сайт: от 1000 до 3000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 3000 
* сайт: до 3000 р.
    * `pay['Maney_min']` = 0
    * `pay['Maney_max']` = 3000
* сайт: 1000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 1000
    
Регулярные выражения: https://www.mongodb.com/docs/manual/reference/operator/query/regex/


In [7]:
arg_ru = [
    {'$and': [{'salary_min': {'$eq': 0}},                              # Рубли
              {'salary_max': {'$gt': pay_level_ru}},
              {'salary_cur': {'$regex': curr_ru, '$options': 'i'}}
             ]
    },              

    {'$and': [{'salary_min': {'$gt': pay_level_ru}}, 
              {'salary_max': {'$eq': 0}},
              {'salary_cur': {'$regex': curr_ru, '$options': 'i'}}
             ]
    },

    {'$and': [{'salary_min': {'$gt': 0}}, 
              {'salary_max': {'$gt': pay_level_ru}},
              {'salary_cur': {'$regex': curr_ru, '$options': 'i'}}
             ]
    },
]

In [8]:
arg_us = [
    {'$and': [{'salary_min': {'$eq': 0}},                              # USD
              {'salary_max': {'$gt': pay_level_us}},
              {'salary_cur': {'$regex': curr_us, '$options': 'i'}}
             ]
    },              

    {'$and': [{'salary_min': {'$gt': pay_level_us}}, 
              {'salary_max': {'$eq': 0}},
              {'salary_cur': {'$regex': curr_us, '$options': 'i'}}
             ]
    },

    {'$and': [{'salary_min': {'$gt': 0}}, 
              {'salary_max': {'$gt': pay_level_us}},
              {'salary_cur': {'$regex': curr_us, '$options': 'i'}}
             ]
    },
]

In [9]:
arg_eu = [
    {'$and': [{'salary_min': {'$eq': 0}},                              # EUR
              {'salary_max': {'$gt': pay_level_eu}},
              {'salary_cur': {'$regex': curr_eu, '$options': 'i'}}
             ]
    },              

    {'$and': [{'salary_min': {'$gt': pay_level_eu}}, 
              {'salary_max': {'$eq': 0}},
              {'salary_cur': {'$regex': curr_eu, '$options': 'i'}}
             ]
    },

    {'$and': [{'salary_min': {'$gt': 0}}, 
              {'salary_max': {'$gt': pay_level_eu}},
              {'salary_cur': {'$regex': curr_eu, '$options': 'i'}}
             ]
    },
]

In [10]:
arg_curr = arg_ru + arg_us + arg_eu
vacancies_selected = vacancies.find({'$or': arg_curr})

#### Выводим результат
Результат выводи на экран в стандартный поток вывода и в файл на диск...

In [11]:
pay_level_ru = 150000  # Задайте зарплату (здесь для удобства)

with open('hh_api_view_MongoDB_show.txt', 'a', encoding='utf-8') as f_out:
    strims = [sys.stdout, f_out]  # выводим в файл и стандартный поток вывода (для удобства)
    
    for strim in strims:
        print(f"PAY-LEVEL:{pay_level_ru}", file=strim)
        
    for i, vacancy in enumerate(vacancies_selected, start=1):
        for strim in strims:
            print(f"{i:3}  {vacancy['salary_min']:6}-{vacancy['salary_max']:6} ",
                  f"{vacancy['salary_cur']:4}  {vacancy['name']}", 
                  file=strim)

PAY-LEVEL:150000
  1    5000-  7000  USD   Head of backend development (Черногория)
  2    3000-  5000  EUR   Senior PHP Developer / PHP программист
  3    2000-  3000  USD   Python Developer (Middle)
  4    3000-  5000  USD   Senior DevOps Engineer (Serbia/Canada)
  5    5000-  7500  USD   Python + R Developer (bioinformatic), New Zealand
  6    5000-  7500  USD   Bioinformatics Developer, New Zealand
  7    5000-  7000  USD   Senior Backend Engineer (Remote)
  8    3000-  4000  USD   Data Analyst (Remote)
  9    2500-  3500  USD   Data Analyst (middle/senior)
 10    2500-  5000  USD   Senior DevOps Engineer
 11    1500-  2500  EUR   Software Developer / smart Cameras
 12    3000-  5000  EUR   DevOps Engineer
 13    4000-  5000  USD   AWS DevOps Engineer (Middle/ Senior) Remote
 14    2000-  4000  USD   Middle Python Developer
 15    2000-  3500  USD   Full-Stack Web-программист
 16    3000-  7000  EUR   Реверс инженер / системный программист
 17    4000-  5000  USD   Team Lead with b

<!--  -->

In [12]:
client.close()

<!--  -->

__P.S.__



Основа представления: 4-е задание этого курса (MongoDB). 

2023-03-01